In [2]:
options(warn=-1)

library(Peptides)

# a function returning per residue average hydrophobicity for a group of sequences of varying length as well as the number of sequences counted at each position
# seqs: a list of sequences
# L_max: take the last L_max residues for very long sequences
# L_min: only consider sequences at least L_min aa long.
# if a sequence is shorter than L_max, fill the beginning with 'x', which will not be used for calculating average hydrophobicity at that position
avg_hydrophobicity_per_position <-function(seqs,L_max,L_min,hydrophobicity_scale){
  H <- rep(0,L_max) # hydrophobicity at each position
  N <- rep(0,L_max) # count of sequence at each position
  i=0
  for(seq in seqs){
    # print progress for every 1000
    i=i+1
    if (i%%1000==0) {
      print(i)
    }
    
    if(nchar(seq) >= L_min){
      seq = paste(paste(rep('x',L_max),collapse=''),seq,sep='')
      seq = substr(seq,nchar(seq)-L_max+1,nchar(seq))
      hi = hydrophobicity(unlist(strsplit(seq,'')),hydrophobicity_scale) 
      H = H + hi 
      N = as.numeric(N) + as.numeric(hi!=0)
    }
  }
  H = H/N
  return(list(H,N))
}

# same as above but look at N-term
avg_hydrophobicity_per_position_nterm <-function(seqs,L_max,L_min,hydrophobicity_scale){
  H <- rep(0,L_max) # hydrophobicity at each position
  N <- rep(0,L_max) # count of sequence at each position
  i=0
  for(seq in seqs){
    # print progress for every 1000
    i=i+1
    if (i%%1000==0) {
      print(i)
    }
    
    if(nchar(seq) >= L_min){
      seq = paste(seq,paste(rep('x',L_max),collapse=''),sep='') # changed
      seq = substr(seq,1,L_max) # changed
      hi = hydrophobicity(unlist(strsplit(seq,'')),hydrophobicity_scale) 
      H = H + hi 
      N = as.numeric(N) + as.numeric(hi!=0)
    }
  }
  H = H/N
  return(list(H,N))
}



In [3]:
# full protein
p=read.table('protein.txt')

# linker
d=read.table('linkers.txt')

# unfolded Cterm
f=read.table('unfolded_cterm.txt')

L_max=100
L_min=100
hydrophobicity_scale = 'Miyazawa' 

CDS_N = avg_hydrophobicity_per_position_nterm(p$V1,L_max,L_min,hydrophobicity_scale)
CDS_C = avg_hydrophobicity_per_position(p$V1,L_max,L_min,hydrophobicity_scale)
unfold_C = avg_hydrophobicity_per_position(f$V1,L_max,L_min,hydrophobicity_scale)

[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] 18000
[1] 19000
[1] 20000
[1] 21000
[1] 22000
[1] 23000
[1] 24000
[1] 25000
[1] 26000
[1] 27000
[1] 28000
[1] 29000
[1] 30000
[1] 31000
[1] 32000
[1] 33000
[1] 34000
[1] 35000
[1] 36000
[1] 37000
[1] 38000
[1] 39000
[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000
[1] 9000
[1] 10000
[1] 11000
[1] 12000
[1] 13000
[1] 14000
[1] 15000
[1] 16000
[1] 17000
[1] 18000
[1] 19000
[1] 20000
[1] 21000
[1] 22000
[1] 23000
[1] 24000
[1] 25000
[1] 26000
[1] 27000
[1] 28000
[1] 29000
[1] 30000
[1] 31000
[1] 32000
[1] 33000
[1] 34000
[1] 35000
[1] 36000
[1] 37000
[1] 38000
[1] 39000
[1] 1000
[1] 2000
[1] 3000
[1] 4000
[1] 5000
[1] 6000
[1] 7000
[1] 8000


In [81]:

L=100
skip_last=3

dev.off()

pdf(paste('N-term-',hydrophobicity_scale,'.pdf',sep=''),width=3,height=3.5)
par(mar=c(4,4,1,1))
x=1:L

y=CDS_N[[1]][(L_max-L+1):L_max]
lo <- loess(y[skip_last:L]~x[skip_last:L])
plot(x,y,ylim=c(5,5.7),pch=16,cex=0.5,col='black',bty='n',ylab='\nAverage hydrophobicity',xlab='Position relative to N-termini\n')
lines(x, predict(lo,x), col='black', lwd=2)

dev.off()

#y=CDS_C[[1]][(L_max-L+1):L_max]
#lo <- loess(y[1:(L-skip_last)]~x[1:(L-skip_last)])
#plot(x,y,ylim=ylim,pch=16,cex=0.2,col='black',bty='n',ylab='\nAverage hydrophobicity',xlab='Position relative to N-termini\n')
#lines(x, predict(lo,x), col='blue', lwd=2)


null device 
          1

null device 
          1

In [85]:
max(CDS_N[[2]])
max(CDS_C[[2]])
max(unfold_C[[2]])


[1] 38933

[1] 38933

[1] 8586

In [77]:
pdf(paste('C-term-unfolded-',hydrophobicity_scale,'.pdf',sep=''),width=3,height=3.5)
par(mar=c(4,4,1,1))
x=-L:-1
y=unfold_C[[1]][(L_max-L+1):L_max]
plot(x[1:(L-skip_last)],y[1:(L-skip_last)],ylim=c(5.19,5.36),bty='l',pch=16,cex=0.5,xlab='Position relative to C-termini',ylab='Average hydrophobicity')
abline(lm(y[1:(L-skip_last)]~x[1:(L-skip_last)]),lwd=2)
ct <- cor.test(y[1:(L-skip_last)],x[1:(L-skip_last)],method='s')
#lo <- loess(y[1:(L-skip_last)]~x[1:(L-skip_last)])
#lines(x, predict(lo,x), col='blue', lwd=2)
legend('topright',bty='n',legend=paste("R = ",format(ct$estimate,digits=3),", P = ",format(ct$p.value,digits=3),sep=''))
dev.off()

PNG 
  2

In [34]:
ct <- cor.test(y[1:(L-skip_last)],x[1:(L-skip_last)],method='s')
ct


	Spearman's rank correlation rho

data:  y[1:(L - skip_last)] and x[1:(L - skip_last)]
S = 222108, p-value = 2.755e-06
alternative hypothesis: true rho is not equal to 0
sample estimates:
       rho 
-0.4603145 


In [44]:
format(ct$p.value ,digits=3)

[1] "2.75e-06"

In [41]:
?format

format {base},R Documentation
x,any R object (conceptually); typically numeric.
trim,"logical; if FALSE, logical, numeric and complex values are right-justified to a common width: if TRUE the leading blanks for justification are suppressed."
digits,"how many significant digits are to be used for numeric and complex x. The default, NULL, uses getOption(""digits""). This is a suggestion: enough decimal places will be used so that the smallest (in magnitude) number has this many significant digits, and also to satisfy nsmall. (For the interpretation for complex numbers see signif.)"
nsmall,the minimum number of digits to the right of the decimal point in formatting real/complex numbers in non-scientific formats. Allowed values are 0 <= nsmall <= 20.
justify,"should a character vector be left-justified (the default), right-justified, centred or left alone. Can be abbreviated."
width,default method: the minimum field width or NULL or 0 for no restriction. AsIs method: the maximum field width for non-character objects. NULL corresponds to the default 12.
na.encode,"logical: should NA strings be encoded? Note this only applies to elements of character vectors, not to numerical, complex nor logical NAs, which are always encoded as ""NA""."
scientific,"Either a logical specifying whether elements of a real or complex vector should be encoded in scientific format, or an integer penalty (see options(""scipen"")). Missing values correspond to the current default penalty."
...,further arguments passed to or from other methods.
"big.mark, big.interval, small.mark, small.interval, decimal.mark, zero.print, drop0trailing",used for prettying (longish) numerical and complex sequences. Passed to prettyNum: that help page explains the details.
